# 1.Preparation
---

## enviroments

In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf


## collection data

In [2]:
import urllib.request

# Set source URL

url_base = 'http://yann.lecun.com/exdb/mnist/'
key_file = {
    'train_img':'train-images-idx3-ubyte.gz',
    'train_label':'train-labels-idx1-ubyte.gz',
    'test_img':'t10k-images-idx3-ubyte.gz',
    'test_label':'t10k-labels-idx1-ubyte.gz'
}

In [3]:
# Determine current directory
pwd = os.getcwd()

# Determine dataset directory which will be created
dataset_dir = os.path.join(pwd, 'dataset')
dataset_dir

'c:\\Users\\redoa\\workspace\\projects\\first_vae\\dataset'

In [6]:
# Creation of dataset directory and check the contents
os.makedirs(dataset_dir, exist_ok=True)
contents = os.listdir('./dataset')
print(contents) # this will show the empty list [] for the first time


['t10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz', 'train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz']


In [5]:
# Download the dataset from source, and check the contents are downloaded
for v in key_file.values():
    file_path = os.path.join(dataset_dir, v)
    if not os.path.exists(file_path):
        urllib.request.urlretrieve(url_base + v, file_path)
        
print(contents)


['t10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz', 'train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz']


 ## Data transformation

In [9]:
# To unzip gz file, use gzip('file name', 'mode e.g. r, w, rb,....')
# 
# To open files, use open() with with statement to open and close the file. Evenif the error occurs, the file will be closed safely.

import gzip

file_path = dataset_dir + '/' + key_file['train_img'] # test with train dataset

with gzip.open(file_path, 'rb') as f:
    data = np.frombuffer(f.read(), np.uint8)
    
len(data)

47040016

data length that we saw must be multiple of mnist data size 28*28. Let's check it

In [8]:
len(data)%(28**2)

16

There are 16 extra data that we do not need. to exclude its, use offset argument.

In [ ]:
with gzip.open(file_path, 'rb') as f:
    data = np.frombuffer(f.read(), np.uint8, offset=16)